<div style="font-size: 200%; font-weight: bold; color: maroon;">Machine learning en SparkR: clustering vía k-means</span></div>
<!-- v. 1.2 - Paulo Villegas, 2019 -->


# Arranque

In [ ]:
# Load SparkR and create a session
library(SparkR)
spark <- sparkR.session( "local[*]" );

# Carga de datos

Usamos de nuevo el dataset de IRIS. Está disponible por defecto en R; lo convertimos en `SparkDataFrame`

In [ ]:
iris.df <- as.DataFrame(iris)

## Separación train/test

In [ ]:
df.list <- randomSplit( iris.df, c(0.8,0.2))

In [ ]:
df.list[]

# Clustering

In [ ]:
# Fit a k-means model with spark.kmeans
kmeansModel <- spark.kmeans(df.list[[1]], ~ Sepal_Length + Sepal_Width + Petal_Length + Petal_Width,
                            k = 3)

In [ ]:
s = summary(kmeansModel)
s

## Análisis

In [ ]:
coef = s[['coefficients']]
storage.mode(coef) <- "numeric"

library(ggplot2)
qplot( Petal_Length, Petal_Width, data=as.data.frame(coef) ) 

In [ ]:
ft <- collect( fitted(kmeansModel) )
names(ft)

In [ ]:
qplot( Petal_Length, Petal_Width, color=as.factor(prediction), data=ft ) + guides( color=FALSE )

In [ ]:
ggplot( mapping=aes(x=Petal_Length, y=Petal_Width) ) + 
   geom_point(data=ft, aes(color=as.factor(prediction))) +
   geom_point(data=as.data.frame(coef), color='purple', size=3) +
   guides( color=FALSE )

## Test

Aplica el clustering al conjunto de test

In [ ]:
kmeansPredictions <- predict( kmeansModel, df.list[[2]])
showDF(kmeansPredictions)

In [ ]:
kmeans_ok = select(kmeansPredictions, 
                   alias(kmeansPredictions$Species == "setosa" & kmeansPredictions$prediction == 1 |
                        kmeansPredictions$Species == "versicolor" & kmeansPredictions$prediction == 0 |
                         kmeansPredictions$Species == "virginica" & kmeansPredictions$prediction == 2, "ok"))

In [ ]:
s = summarize(kmeans_ok, n(kmeans_ok$ok))

In [ ]:
take(s, 1)

In [ ]:
count(s)